In [63]:
import numpy as np

In [64]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [65]:
def calculate_gradient(theta, X, y):
    m = y.size      # This is the number of instances
    return (X.T @ (sigmoid(X @ theta) - y)) / m

In [66]:
def gradient_descent(X, y, alpha=0.1, num_iterations=100, tol=1e-7):
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]

    theta = np.zeros(X_bias.shape[1])

    for i in range(num_iterations):
        gradient = calculate_gradient(theta, X_bias, y)
        theta -= alpha * gradient

        if np.linalg.norm(gradient) < tol:
            break

    return theta

In [67]:
def predict_proba(X, theta):
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]

    return sigmoid(X_bias @ theta)


In [68]:
def predict(X, theta, threshold=0.5):
    return (predict_proba(X, theta) >= threshold).astype(int)

In [69]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [70]:
iris_data = load_iris()
X, y = iris_data.data, iris_data.target

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [72]:
# Training one classifier for each class using the One Vs Rest method (class 0 or non-class 0 and so on)
all_thetas = []
num_classes = len(np.unique(y))     # There are 3 unique classes in the Iris dataset

print(f"Training {num_classes} separate binary classifiers")

for i in range(num_classes):
    # Create a temporary target vector for this specific class
    y_train_specific = (y_train == i).astype(int)

    print(f"    Training classifier for class {i} ({iris_data.target_names[i]}) vs the rest")

    # Using the gradient descent function to train the binary classifier
    theta_hat = gradient_descent(X_train_scaled, y_train_specific, alpha=0.1, num_iterations=1000)

    # Store the trained parameters
    all_thetas.append(theta_hat)

print("Training complete")

Training 3 separate binary classifiers
    Training classifier for class 0 (setosa) vs the rest
    Training classifier for class 1 (versicolor) vs the rest
    Training classifier for class 2 (virginica) vs the rest
Training complete


In [73]:
def predict_ovr(X, thetas):
    probabilities = np.array([predict_proba(X, theta) for theta in thetas]).T

    return np.argmax(probabilities, axis=1)

In [74]:
# Predictions on the test set
y_pred = predict_ovr(X_test_scaled, all_thetas)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

# Classification report for a more detailed look
print("\nClassification report")
print(classification_report(y_test, y_pred, target_names=iris_data.target_names))


Model Accuracy: 0.9667

Classification report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         8
  versicolor       1.00      0.83      0.91         6
   virginica       0.94      1.00      0.97        16

    accuracy                           0.97        30
   macro avg       0.98      0.94      0.96        30
weighted avg       0.97      0.97      0.97        30

